### TABLE CREATION

In [ ]:
from CreateTable import create_df
import pickle
import pandas as pd

# data_test = create_df("DamasconeC/together/*_CPXE_*.csv", symm_v = True, angle=-1, offset=(52, 51))  # , symmetric = True
# data_train = create_df("DamasconeA/data2/*_CPXE_*.csv", offset=(49, 51), angle=1)
# data_train = create_df("DamasconeB/together/*_CPXE_*.csv")
data_train = pd.read_pickle("DamasconeA2.pkl")

data_validation = pd.read_pickle("DamasconeA.pkl")
# data_train2 = data_train2.drop(columns=["Row"])

data_test = pd.read_pickle("DamasconeB.pkl")
# data_test = pd.read_pickle("C_norm_ss.pkl")

# to be handled properly TODO


In [ ]:
import numpy as np

data_train['stiffness_to_relaxation'] = data_train['Stiffness'] / data_train['force_relaxation']
data_train['oscillation_to_max_force'] = data_train['force_oscillation'] / data_train['force_max']

data_test['stiffness_to_relaxation'] = data_test['Stiffness'] / data_test['force_relaxation']
data_test['oscillation_to_max_force'] = data_test['force_oscillation'] / data_test['force_max']

data_validation['stiffness_to_relaxation'] = data_validation['Stiffness'] / data_validation['force_relaxation']
data_validation['oscillation_to_max_force'] = data_validation['force_oscillation'] / data_validation['force_max']

data_train = data_train.dropna()
data_test = data_test.dropna()
data_validation = data_validation.dropna()



In [ ]:
from helper import feature_list2

# Create dataset with selected features (no posx, posy)
# feature_list_time = [ 'Stiffness', 'Upstroke', 'P_ss']
# feature_list_hysteresis = ['peak_position','loading_energy']
# feature_list = feature_list_time + feature_list_hysteresis
feature_list = feature_list2
#
# to_remove = ['segment2_slope', 'segment2_force_std',
#              'segment2_skew', 'segment3_skew', 'Downstroke', 'poly3_coef0',
#              'poly5_coef0', 'poly5_coef1', 'poly5_coef2', 'poly5_coef3', 'poly5_coef4', 'Entropy', "Tau", 'P_ss',
#              'offset', 'force_relaxation', 'force_ratio_75_25', 'hysteresis_area'
#              ]
#

to_remove = ['poly3_coef0', 'poly3_coef1', 'poly3_coef2', 'poly3_coef3',
             'poly4_coef0', 'poly4_coef1', 'poly4_coef2', 'poly4_coef3', 'poly4_coef4',
             'poly5_coef0', 'poly5_coef1', 'poly5_coef2', 'poly5_coef3', 'poly5_coef4',
             # Segmentation features
             'segment2_slope', 'segment3_slope', 'segment2_force_std', 'segment3_force_std',
             'segment2_skew', 'segment3_skew']

feature_list_3 = feature_list = [x for x in feature_list2 + ['stiffness_to_relaxation', 'oscillation_to_max_force'] if
                x not in to_remove]
# feature_list = ['loading_unloading_area_ratio', 'peak_width', 'stiffness_to_relaxation', 'Stiffness']
# feature_list = ["Stiffness", "Upstroke", "Downstroke", "P_ss"] #, "P_ss", "loading_energy"]peak_position

# feature_list = ['poly4_coef4', 'poly4_coef3', 'poly5_coef0', 'loading_unloading_area_ratio', 'Stiffness', 'quartic_coefficient', 'Upstroke']

### PLOTS

### SPATIAL SMOOTHING

In [ ]:
from scipy.interpolate import NearestNDInterpolator
from scipy.ndimage import median_filter, gaussian_filter
import numpy as np
import pandas as pd
from helper import *

smoothing_config = {
    'Entropy': 'gaussian',
}  # Dictionary 'feature-name' : type of smoothing (set if you want to test other method of smoothing instead of median)


def apply_smoothing(grid_z, method='median'):
    if method == 'median':
        return median_filter(grid_z, size=3, mode='reflect')
    elif method == 'gaussian':
        return gaussian_filter(grid_z, sigma=2)
    elif method == 'diffusion':
        # Simple anisotropic diffusion
        def diffusion_step(img, kappa=50):
            # Compute image gradients
            dy, dx = np.gradient(img)

            # Compute diffusion coefficients
            diff_coef_x = 1 / (1 + (dx / kappa) ** 2)
            diff_coef_y = 1 / (1 + (dy / kappa) ** 2)

            # Compute diffusion
            diff_x = np.zeros_like(img)
            diff_y = np.zeros_like(img)

            diff_x[1:-1, 1:-1] = diff_coef_x[1:-1, 1:-1] * (img[1:-1, 2:] - img[1:-1, 1:-1])
            diff_y[1:-1, 1:-1] = diff_coef_y[1:-1, 1:-1] * (img[2:, 1:-1] - img[1:-1, 1:-1])

            return img + 0.25 * (diff_x + diff_y)

        # Apply multiple diffusion steps
        iterations = 20
        img = grid_z.copy()
        for i in range(iterations):
            img = diffusion_step(img)
        return img

    else:
        raise ValueError(f"Smoothing method not implemented: {method}")


def get_grid_bounds(df):
    x_min, x_max = int(df["posx"].min()), int(df["posx"].max())
    y_min, y_max = int(df["posy"].min()), int(df["posy"].max())
    grid_shape = (y_max - y_min + 1, x_max - x_min + 1)
    return x_min, y_min, grid_shape


# Smoothing function for a single dataframe
def smooth_subset(subset_df, x_min, y_min, grid_shape):
    smoothed_subset = pd.DataFrame(index=subset_df.index)

    for feature in feature_list:
        # Initialize grid with NaNs
        grid_z = np.full(grid_shape, np.nan, dtype=np.float32)

        # Map each data point to the grid
        for _, row in subset_df.iterrows():
            x_idx = int(row["posx"]) - x_min
            y_idx = int(row["posy"]) - y_min
            grid_z[y_idx, x_idx] = row[feature]

        # Interpolate missing values
        yy, xx = np.indices(grid_z.shape)
        valid_mask = ~np.isnan(grid_z)

        if np.any(~valid_mask):
            interpolator = NearestNDInterpolator(
                np.column_stack((yy[valid_mask], xx[valid_mask])),
                grid_z[valid_mask]
            )
            grid_z = interpolator(yy, xx)

        # Apply feature-specific smoothing
        methods = smoothing_config.get(feature,
                                       'median')  # Get the designed methods, if None select defaul 'median' method
        if isinstance(methods, list):
            for method in methods:
                grid_z = apply_smoothing(grid_z, method=method)
        else:
            grid_z = apply_smoothing(grid_z, method=methods)

        # Map smoothed grid back to DataFrame
        smoothed_subset[feature] = [
            grid_z[int(row["posy"]) - y_min, int(row["posx"]) - x_min]
            for _, row in subset_df.iterrows()
        ]

    # Add back metadata columns
    smoothed_subset[['label', 'posx', 'posy']] = subset_df[['label', 'posx', 'posy']]
    return smoothed_subset


if smoothing_config is None:
    smoothing_config = {feature: 'median' for feature in feature_list}

# Compute grid bounds and smooth each subset
test_x_min, test_y_min, test_grid_shape = get_grid_bounds(data_test)
train_x_min, train_y_min, train_grid_shape = get_grid_bounds(data_train)
val_x_min, val_y_min, val_grid_shape = get_grid_bounds(data_validation)

# Smooth each subset SEPARATELY
smoothed_test = smooth_subset(data_test, test_x_min, test_y_min, test_grid_shape)
smoothed_train = smooth_subset(data_train, train_x_min, train_y_min, train_grid_shape)
smoothed_validation = smooth_subset(data_validation, val_x_min, val_y_min, val_grid_shape)

smoothed_df = pd.concat([smoothed_test, smoothed_train],
                        ignore_index=True)  # For visualization in the next sections only

### DB SPLITTING

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
from imblearn.over_sampling import ADASYN
import warnings
from sklearn.exceptions import UndefinedMetricWarning
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

# Display class distribution
X = smoothed_df[feature_list + ["posx", "posy"]]
y = smoothed_df['label']

# train_df = pd.concat([train_df1, train_df2], ignore_index=True)
# smoothed_train = pd.concat([smoothed_train1, smoothed_train2], ignore_index=True)

# Training distribution:
class_distribution = Counter(smoothed_train['label'])
print("Class distribution in the dataset:")
for label, count in sorted(class_distribution.items()):
    print(f"Label {label}: {count} samples ({count / len(y) * 100:.2f}%)")

# Test distribution:
class_distribution = Counter(smoothed_test['label'])
print("Class distribution in the dataset:")
for label, count in sorted(class_distribution.items()):
    print(f"Label {label}: {count} samples ({count / len(y) * 100:.2f}%)")

# Prepare data for classification
X_train = smoothed_train[feature_list + ["posx", "posy"]]
y_train = smoothed_train['label']

X_test = smoothed_test[feature_list + ["posx", "posy"]]
y_test = smoothed_test['label']

X_validation = smoothed_validation[feature_list + ["posx", "posy"]]
y_validation = smoothed_validation['label']

# Create the figure and axes: 2 rows (train/test), then 3 histograms
fig, axs = plt.subplots(2, 4, figsize=(22, 14), gridspec_kw={'width_ratios': [3, 1, 1, 1]})

# Set style
sns.set(style="whitegrid")

# Create consistent color map using turbo
unique_labels = sorted(data_train['label'].unique())  # or union of train + test if needed
palette_colors = sns.color_palette("turbo", n_colors=len(unique_labels))
label_color_dict = {label: color for label, color in zip(unique_labels, palette_colors)}

# Plot train
sns.scatterplot(
    data=data_train,
    x='posx',
    y='posy',
    hue='label',
    palette=label_color_dict,  # Use fixed mapping
    ax=axs[0, 0]
)
axs[0, 0].set_title("Train Set Distribution (posx vs posy)")
axs[0, 0].set_xlabel("Position X (posx)")
axs[0, 0].set_ylabel("Position Y (posy)")
axs[0, 0].legend(title='Label', bbox_to_anchor=(1.05, 1), loc='upper left')

# Plot test with same label-color mapping
sns.scatterplot(
    data=data_test,
    x='posx',
    y='posy',
    hue='label',
    palette=label_color_dict,  # Same dictionary ensures color consistency
    ax=axs[1, 0]
)
axs[1, 0].set_title("Test Set Distribution (posx vs posy)")
axs[1, 0].set_xlabel("Position X (posx)")
axs[1, 0].set_ylabel("Position Y (posy)")
axs[1, 0].legend(title='Label', bbox_to_anchor=(1.05, 1), loc='upper left')

# --- Histograms / Class Distribution Plots ---
plot_class_distribution(smoothed_df['label'], axs[0, 1], "Class Dist. (Entire)")
plot_class_distribution(y_train, axs[0, 2], "Class Dist. (Train)")
plot_class_distribution(y_test, axs[0, 3], "Class Dist. (Test)")

# Hide the unused bottom row histograms
for ax in axs[1, 1:]:
    ax.axis('off')

plt.tight_layout()
plt.show()

# # Drop posx and posy columns from the training and testing sets after plotting
# X_train = X_train.drop(columns=['posx', 'posy'])
# X_test = X_test.drop(columns=['posx', 'posy'])

### Binary Classifier

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_train1 = StandardScaler()
scaler_train2 = StandardScaler()
scaler_test = StandardScaler()

X_train_selected = X_train[feature_list]
X_test_selected = X_test[feature_list]
X_validation_selected = X_validation[feature_list]

# X_train_selected_1 = X_train_selected[X_train["posy"] <= 140]
# X_train_selected_2 = X_train_selected[X_train["posy"] > 140]

X_train_scaled = scaler_train1.fit_transform(X_train_selected)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_list, index=X_train_selected.index)

X_validation_scaled = scaler_train1.fit_transform(X_validation_selected)
X_validation_scaled = pd.DataFrame(X_validation_scaled, columns=feature_list, index=X_validation_selected.index)

# X_train_scaled2 = scaler_train2.fit_transform(X_train_selected_2)

# X_train_scaled1 = pd.DataFrame(X_train_scaled1, columns=feature_list, index=X_train_selected_1.index)
# X_train_scaled2 = pd.DataFrame(X_train_scaled2, columns=feature_list, index=X_train_selected_2.index)
# X_train_scaled = pd.concat([X_train_scaled1, X_train_scaled2]).sort_index()

X_test_scaled = scaler_test.fit_transform(X_test_selected)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_list, index=X_test_selected.index)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
from sklearn.feature_selection import mutual_info_classif


def calculate_mutual_info(X_data, y_data, feature_list, random_state=42):
    """Calculate mutual information scores for features."""
    return mutual_info_classif(X_data[feature_list], y_data, random_state=random_state)


def calculate_wasserstein_distances(train_data, validation_data, feature_list):
    """Calculate Wasserstein distance between train and validation for each feature."""
    distances = []

    for feature in feature_list:
        distance = wasserstein_distance(train_data[feature], validation_data[feature])
        distances.append({'feature': feature, 'wasserstein_distance': distance})

    return pd.DataFrame(distances)


def plot_feature_importance(mi_df, distance_df, figsize=(24, 9)):
    """Plot mutual information and Wasserstein distance for features."""
    fig, axes = plt.subplots(1, 2, figsize=figsize, sharey=True)

    # Ensure both dataframes have the same feature ordering
    # Sort by mutual information scores
    mi_df = mi_df.sort_values('smoothed', ascending=True)

    # Merge with distance_df to maintain the same order
    merged_df = mi_df[['feature']].merge(distance_df, on='feature')

    # Plot Mutual Information
    bar_width = 0.45
    y_positions = np.arange(len(mi_df))
    ax = axes[0]

    ax.barh(y_positions, mi_df['smoothed'], height=bar_width, color='blue',
            alpha=0.6, label='Train')
    ax.barh(y_positions + bar_width, mi_df['test'], height=bar_width, color='green',
            alpha=0.6, label='Validation')

    ax.set_yticks(y_positions)
    ax.set_yticklabels(mi_df['feature'])
    ax.set_title('Mutual Information Scores', fontsize=16)
    ax.set_xlabel('Mutual Info Score', fontsize=12)
    ax.set_ylabel('')
    ax.legend()

    # Plot Wasserstein Distance - use the merged dataframe to keep order consistent
    sns.barplot(x='wasserstein_distance', y='feature', data=merged_df,
                palette='mako', ax=axes[1], hue='feature', legend=False)
    axes[1].set_title('Train vs Test Wasserstein Distance', fontsize=16)
    axes[1].set_xlabel('Distance', fontsize=12)
    axes[1].set_ylabel('')
    axes[1].set_xlim(0, 1)

    plt.tight_layout()
    return fig


def select_features(mi_df, distance_df, mi_threshold=0.5, distance_threshold=0.12):
    """Select features based on mutual information and Wasserstein distance thresholds."""
    combined_df = mi_df.merge(distance_df, on='feature')

    selected_features = combined_df[
        (combined_df['smoothed'] > mi_threshold) &
        (combined_df['test'] > mi_threshold) &
        (combined_df['wasserstein_distance'] < distance_threshold)
        ]

    return selected_features


def main(X_train_scaled, y_train, X_validation_scaled, y_validation, feature_list, selected_labels, mi_threshold=0.5,
         distance_threshold=0.1):
    """Main function to perform feature selection analysis."""
    # Calculate mutual information for train and validation sets
    mi_scores_train = calculate_mutual_info(X_train_scaled, y_train, feature_list)
    mi_scores_val = calculate_mutual_info(X_validation_scaled, y_validation, feature_list)

    # Create DataFrame for mutual information scores
    mi_df = pd.DataFrame({
        'feature': feature_list,
        'smoothed': mi_scores_train,
        'test': mi_scores_val,
    }).sort_values('smoothed', ascending=True)

    train_mask = y_train.isin(selected_labels)
    X_train_filtered = X_train_scaled[train_mask]

    # Filter validation data
    test_mask = y_validation.isin(selected_labels)
    X_val_filtered = X_validation_scaled[test_mask]

    # Use the filtered data for distance calculation
    distance_df = calculate_wasserstein_distances(
        X_train_filtered, X_val_filtered, feature_list
    )

    # Plot feature importance
    plot_feature_importance(mi_df, distance_df)
    plt.show()

    # Select features based on thresholds
    selected = select_features(mi_df, distance_df, mi_threshold, distance_threshold)

    # Output selected features
    print(f"✅ Features with MI (train & validation > {mi_threshold}) and "
          f"Wasserstein < {distance_threshold}:")
    print(selected['feature'].tolist())

    return selected['feature'].tolist()


selected_features = main(
    X_train_scaled=X_train_scaled,
    y_train=y_train,
    X_validation_scaled=X_validation_scaled,
    y_validation=y_validation,
    feature_list=feature_list,
    selected_labels=[2, 3, 4, 5],
    mi_threshold=0.53,
    distance_threshold=0.08
)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance, ks_2samp, entropy
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import pairwise_distances
import warnings

warnings.filterwarnings('ignore')


def calculate_mutual_info(X_data, y_data, feature_list, random_state=42):
    """Calculate mutual information scores for features."""
    return mutual_info_classif(X_data[feature_list], y_data, random_state=random_state)


def calculate_wasserstein_distances(train_data, validation_data, feature_list):
    """Calculate Wasserstein distance between train and validation for each feature."""
    distances = []

    for feature in feature_list:
        distance = wasserstein_distance(train_data[feature], validation_data[feature])
        distances.append({'feature': feature, 'wasserstein_distance': distance})

    return pd.DataFrame(distances)


def jensen_shannon_distance(p, q):
    """Calculate Jensen-Shannon distance between two distributions."""
    # Convert to probability distributions
    p = np.asarray(p) / np.sum(p)
    q = np.asarray(q) / np.sum(q)

    # Calculate M
    m = 0.5 * (p + q)

    # Calculate JS divergence
    js_divergence = 0.5 * (entropy(p, m) + entropy(q, m))

    # Convert to distance (square root of divergence)
    return np.sqrt(js_divergence)


def calculate_distribution_metrics(train_data, validation_data, feature_list):
    """Calculate multiple distribution metrics between train and validation sets."""
    metrics = []

    for feature in feature_list:
        # Wasserstein distance (Earth Mover's Distance)
        w_distance = wasserstein_distance(train_data[feature], validation_data[feature])

        # Kolmogorov-Smirnov test
        ks_stat, ks_pval = ks_2samp(train_data[feature], validation_data[feature])

        # Jensen-Shannon distance
        # First, we need to create histograms to compare distributions
        hist_bins = 30
        train_hist, _ = np.histogram(train_data[feature], bins=hist_bins, density=True)
        val_hist, _ = np.histogram(validation_data[feature], bins=hist_bins, density=True)

        # Add small epsilon to avoid divide by zero issues
        train_hist = train_hist + 1e-10
        val_hist = val_hist + 1e-10

        js_dist = jensen_shannon_distance(train_hist, val_hist)

        metrics.append({
            'feature': feature,
            'wasserstein_distance': w_distance,
            'ks_statistic': ks_stat,
            'ks_pvalue': ks_pval,
            'jensen_shannon_distance': js_dist
        })

    return pd.DataFrame(metrics)


def calculate_class_distribution_metrics(X_train, y_train, X_validation, y_validation, feature_list, selected_classes):
    """Calculate distribution metrics for each selected class."""
    class_metrics = {}

    for cls in selected_classes:
        # Filter data for this class
        X_train_cls = X_train[y_train == cls]
        X_val_cls = X_validation[y_validation == cls]

        # Calculate metrics for this class
        metrics_df = calculate_distribution_metrics(X_train_cls, X_val_cls, feature_list)
        class_metrics[cls] = metrics_df

    return class_metrics


def plot_feature_importance(mi_df, metrics_df, figsize=(24, 12)):
    """Plot mutual information and distribution metrics for features."""
    fig, axes = plt.subplots(2, 2, figsize=figsize)

    # Ensure both dataframes have the same feature ordering
    mi_df = mi_df.sort_values('smoothed', ascending=False)

    # Merge to maintain the same order
    merged_df = mi_df[['feature']].merge(metrics_df, on='feature')

    # Plot Mutual Information
    sns.barplot(x='smoothed', y='feature', data=mi_df.head(15),
                color='blue', alpha=0.7, ax=axes[0, 0])
    sns.barplot(x='test', y='feature', data=mi_df.head(15),
                color='green', alpha=0.7, ax=axes[0, 1])

    axes[0, 0].set_title('Mutual Information (Train)', fontsize=14)
    axes[0, 1].set_title('Mutual Information (Validation)', fontsize=14)

    # Plot distribution metrics
    sns.barplot(x='wasserstein_distance', y='feature', data=merged_df.head(15),
                palette='mako', ax=axes[1, 0])
    sns.barplot(x='jensen_shannon_distance', y='feature', data=merged_df.head(15),
                palette='rocket', ax=axes[1, 1])

    axes[1, 0].set_title('Wasserstein Distance (Train vs Validation)', fontsize=14)
    axes[1, 1].set_title('Jensen-Shannon Distance (Train vs Validation)', fontsize=14)

    plt.tight_layout()
    return fig


def plot_class_distribution_metrics(class_metrics, selected_classes, top_features=10, figsize=(15, 10)):
    """Plot distribution metrics across classes for top features."""
    # Prepare data for plotting
    plot_data = []

    # Get the top features based on average Wasserstein distance across classes
    feature_avg_metrics = {}

    for feature in class_metrics[selected_classes[0]]['feature'].unique():
        feature_metrics = []
        for cls in selected_classes:
            cls_df = class_metrics[cls]
            feature_row = cls_df[cls_df['feature'] == feature]
            if not feature_row.empty:
                feature_metrics.append(feature_row['wasserstein_distance'].values[0])

        feature_avg_metrics[feature] = np.mean(feature_metrics)

    # Sort features by average Wasserstein distance
    sorted_features = sorted(feature_avg_metrics.items(), key=lambda x: x[1])
    top_features_list = [f[0] for f in sorted_features[:top_features]]

    # Create data for plotting
    for feature in top_features_list:
        for cls in selected_classes:
            cls_df = class_metrics[cls]
            feature_row = cls_df[cls_df['feature'] == feature]

            if not feature_row.empty:
                plot_data.append({
                    'feature': feature,
                    'class': cls,
                    'wasserstein_distance': feature_row['wasserstein_distance'].values[0],
                    'jensen_shannon_distance': feature_row['jensen_shannon_distance'].values[0]
                })

    plot_df = pd.DataFrame(plot_data)

    # Create plots
    fig, axes = plt.subplots(1, 2, figsize=figsize)

    # Plot Wasserstein distance by class
    sns.barplot(x='wasserstein_distance', y='feature', hue='class',
                data=plot_df, ax=axes[0], palette='viridis')
    axes[0].set_title('Wasserstein Distance by Class', fontsize=14)
    axes[0].legend(title='Class')

    # Plot Jensen-Shannon distance by class
    sns.barplot(x='jensen_shannon_distance', y='feature', hue='class',
                data=plot_df, ax=axes[1], palette='viridis')
    axes[1].set_title('Jensen-Shannon Distance by Class', fontsize=14)
    axes[1].legend(title='Class')

    plt.tight_layout()
    return fig


def visualize_feature_distributions(X_train, y_train, X_validation, y_validation,
                                    selected_features, selected_classes,
                                    n_features=4, figsize=(16, 12)):
    """Visualize the distributions of selected features across classes."""
    if len(selected_features) > n_features:
        selected_features = selected_features[:n_features]

    fig, axes = plt.subplots(len(selected_features), len(selected_classes), figsize=figsize)

    for i, feature in enumerate(selected_features):
        for j, cls in enumerate(selected_classes):
            ax = axes[i, j] if len(selected_features) > 1 else axes[j]

            # Filter data for the class
            train_data = X_train[y_train == cls][feature]
            val_data = X_validation[y_validation == cls][feature]

            # Plot distributions
            sns.histplot(train_data, kde=True, color='blue', alpha=0.5,
                         ax=ax, label='Train', stat='density')
            sns.histplot(val_data, kde=True, color='green', alpha=0.5,
                         ax=ax, label='Validation', stat='density')

            # Calculate Wasserstein distance for this feature and class
            w_dist = wasserstein_distance(train_data, val_data)

            ax.set_title(f"Class {cls}, Feature: {feature}\nWasserstein = {w_dist:.4f}", fontsize=10)

            if i == 0:
                ax.legend()

            if i == len(selected_features) - 1:
                ax.set_xlabel(feature)
            else:
                ax.set_xlabel('')

            if j == 0:
                ax.set_ylabel('Density')
            else:
                ax.set_ylabel('')

    plt.tight_layout()
    return fig


def select_features_advanced(mi_df, metrics_df, class_metrics, selected_classes,
                             mi_threshold=0.5, wasserstein_threshold=0.12,
                             jensen_shannon_threshold=0.3,
                             max_class_deviation=0.05):
    """
    Select features based on multiple criteria:
    1. High mutual information in both train and validation
    2. Low distribution distances between train and validation
    3. Consistent distribution across all selected classes
    """
    # Step 1: Filter by mutual information
    mi_selected = mi_df[(mi_df['smoothed'] > mi_threshold) &
                        (mi_df['test'] > mi_threshold)]

    # Step 2: Filter by overall distribution metrics
    dist_selected = metrics_df[(metrics_df['wasserstein_distance'] < wasserstein_threshold) &
                               (metrics_df['jensen_shannon_distance'] < jensen_shannon_threshold)]

    # Combine the filters
    combined_selected = mi_selected[['feature']].merge(dist_selected[['feature']], on='feature')

    # Step 3: Filter by class consistency
    consistent_features = []

    for feature in combined_selected['feature']:
        # Check if feature is consistent across all classes
        class_wasserstein_values = []
        class_js_values = []

        for cls in selected_classes:
            cls_df = class_metrics[cls]
            feature_row = cls_df[cls_df['feature'] == feature]

            if not feature_row.empty:
                class_wasserstein_values.append(feature_row['wasserstein_distance'].values[0])
                class_js_values.append(feature_row['jensen_shannon_distance'].values[0])

        # Calculate coefficient of variation (std / mean) as a measure of consistency
        wasserstein_cv = np.std(class_wasserstein_values) / (np.mean(class_wasserstein_values) + 1e-10)
        js_cv = np.std(class_js_values) / (np.mean(class_js_values) + 1e-10)

        # Check if the feature is consistent across classes
        if wasserstein_cv < max_class_deviation and js_cv < max_class_deviation:
            consistent_features.append(feature)

    # Create final selection
    final_selection = combined_selected[combined_selected['feature'].isin(consistent_features)]

    return final_selection['feature'].tolist()


def main(X_train_scaled, y_train, X_validation_scaled, y_validation, feature_list,
         selected_classes, mi_threshold=0.5, wasserstein_threshold=0.1,
         jensen_shannon_threshold=0.3, class_deviation_threshold=0.3):
    """Enhanced main function to perform feature selection analysis."""
    # print(f"Analyzing feature distributions for classes: {selected_classes}")

    # Filter data for selected classes
    train_mask = y_train.isin(selected_classes)
    X_train_filtered = X_train_scaled[train_mask]
    y_train_filtered = y_train[train_mask]

    test_mask = y_validation.isin(selected_classes)
    X_val_filtered = X_validation_scaled[test_mask]
    y_val_filtered = y_validation[test_mask]

    # Calculate mutual information for train and validation sets
    mi_scores_train = calculate_mutual_info(X_train_filtered, y_train_filtered, feature_list)
    mi_scores_val = calculate_mutual_info(X_val_filtered, y_val_filtered, feature_list)

    # Create DataFrame for mutual information scores
    mi_df = pd.DataFrame({
        'feature': feature_list,
        'smoothed': mi_scores_train,
        'test': mi_scores_val,
    })

    # Calculate overall distribution metrics
    metrics_df = calculate_distribution_metrics(X_train_filtered, X_val_filtered, feature_list)

    # Calculate class-specific distribution metrics
    class_metrics = calculate_class_distribution_metrics(
        X_train_filtered, y_train_filtered,
        X_val_filtered, y_val_filtered,
        feature_list, selected_classes
    )

    # # Plot feature importance
    # print("Plotting overall feature importance...")
    # plot_feature_importance(mi_df, metrics_df)
    # plt.show()
    #
    # # Plot class-specific metrics
    # print("Plotting class-specific distribution metrics...")
    # plot_class_distribution_metrics(class_metrics, selected_classes)
    # plt.show()

    # Select features based on enhanced criteria
    selected_features = select_features_advanced(
        mi_df, metrics_df, class_metrics, selected_classes,
        mi_threshold, wasserstein_threshold,
        jensen_shannon_threshold, class_deviation_threshold
    )
    #
    # # Visualize distributions of selected features
    # print("Visualizing distributions of selected features across classes...")
    # if len(selected_features) > 0:
    #     visualize_feature_distributions(
    #         X_train_filtered, y_train_filtered,
    #         X_val_filtered, y_val_filtered,
    #         selected_features, selected_classes
    #     )
    #     plt.show()
    #
    # # Output selected features
    # print(f"\n✅ Selected Features ({len(selected_features)}):")
    # print(f"  - MI threshold: {mi_threshold}")
    # print(f"  - Wasserstein threshold: {wasserstein_threshold}")
    # print(f"  - Jensen-Shannon threshold: {jensen_shannon_threshold}")
    # print(f"  - Class deviation threshold: {class_deviation_threshold}")
    #
    # if len(selected_features) > 0:
    #     for i, feature in enumerate(selected_features, 1):
    #         print(f"  {i}. {feature}")
    # else:
    #     print("  No features met all criteria. Consider relaxing thresholds.")

    return selected_features


# Example usage
selected_features = main(
    X_train_scaled=X_train_scaled,
    y_train=y_train,
    X_validation_scaled=X_validation_scaled,
    y_validation=y_validation,
    feature_list=feature_list_3,
    selected_classes=[2, 3, 4, 5],
    mi_threshold=0.3,
    wasserstein_threshold=0.275,
    jensen_shannon_threshold=0.25,
    class_deviation_threshold=110
)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score
from tqdm import tqdm


def find_best_smoothing_params(
        df,
        features_to_smooth,
        class_label_col,
        bandwidth_values,
        center_emphasis_values,
):
    """
    For each (bw,ce) in the grid, smooth the df, then compute
    silhouette_score on the smoothed feature matrix (only the smoothed
    features), using the true class labels in `class_label_col`.
    Returns a DataFrame of scores and the best (bw, ce).
    """
    results = []
    true_labels = df[class_label_col].values

    for bw in tqdm(bandwidth_values, desc="bandwidth"):
        for ce in center_emphasis_values:
            # 1) smooth
            selected_features = main(
                X_train_scaled=X_train_scaled,
                y_train=y_train,
                X_validation_scaled=X_validation_scaled,
                y_validation=y_validation,
                feature_list=feature_list_3,
                selected_classes=[2, 3, 4, 5],
                mi_threshold=0.3,
                wasserstein_threshold=bw,
                jensen_shannon_threshold=ce,
                class_deviation_threshold=110
            )

            X = df[selected_features].values

            # 2) silhouette (note: requires at least 2 points per class)
            try:
                score = silhouette_score(X, true_labels)
            except ValueError:
                score = np.nan

            results.append({
                'w_th': bw,
                'js_th': ce,
                'n_features': len(selected_features),
                'silhouette': score
            })

    results_df = pd.DataFrame(results)
    # pick the best
    best = results_df.loc[results_df['silhouette'].idxmax()]
    return results_df, best['w_th'], best['js_th'], best['silhouette']

bw_vals =  np.arange(0, 0.3, 0.025).tolist()
ce_vals = np.arange(0, 0.3, 0.025).tolist()
features = [f for f in feature_list if f not in ['posx', 'posy', 'label', 'class']]

df_all = pd.concat([X_train_scaled, X_validation_scaled], axis=0, ignore_index=True)
df_all['class'] = pd.concat([y_train, y_validation], axis=0, ignore_index=True)

# df_all = hard_spots_validation
# df_all['class'] = y_validation_hard

results_df, best_bw, best_ce, best_score = find_best_smoothing_params(
    df=df_all,
    features_to_smooth=features,
    class_label_col='class',
    bandwidth_values=bw_vals,
    center_emphasis_values=ce_vals
)

print("Best params:", best_bw, best_ce, ". silhouette:", best_score)
results_df = results_df.dropna()

# keeps the best silhouette score for equal number of features
results_df = results_df.loc[results_df.groupby('n_features')['silhouette'].idxmax()].reset_index(drop=True)

results_df

In [ ]:
results_df = results_df.dropna()

# keeps the best silhouette score for equal number of features
results_df = results_df.loc[results_df.groupby('n_features')['silhouette'].idxmax()].reset_index(drop=True)

results_df

In [ ]:
feature_list = selected_features

In [ ]:
from plotter import plot_features

X_train_nz = X_train_scaled.copy()
X_train_nz["label"] = y_train
X_train_nz["posy"] = X_train["posy"]
X_train_nz["posx"] = X_train["posx"]

X_test_new_nz = X_test_scaled.copy()
X_test_new_nz["label"] = y_test
X_test_new_nz["posx"] = X_test["posx"]
X_test_new_nz["posy"] = X_test["posy"]

X_val_new_nz = X_validation_scaled.copy()
X_val_new_nz["label"] = y_validation
X_val_new_nz["posx"] = X_validation["posx"]
X_val_new_nz["posy"] = X_validation["posy"]

# plot_features(X_train_nz, X_test_new_nz, feature_list)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, balanced_accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE

# Create dataset with selected features (excluding posx, posy)
# X_train_selected = X_train[feature_list]
# X_test_selected = X_test[feature_list]

X_train_selected = X_train_scaled[feature_list]
X_test_selected = X_test_scaled[feature_list]
X_validation_selected = X_validation_scaled[feature_list]

# Convert labels to binary classification (0 vs all)
y_train_binary = np.where(y_train == 0, 0, 1)
y_test_binary = np.where(y_test == 0, 0, 1)

# Apply ADASYN for class imbalance handling
adasyn = ADASYN(sampling_strategy='auto', random_state=42, n_neighbors=10)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_selected, y_train_binary)
# X_train_adasyn, y_train_multi = X_train_selected, y_train

# Display new class distribution after ADASYN
adasyn_class_distribution = Counter(y_train_adasyn)
print("\nClass distribution after ADASYN:")
for label, count in sorted(adasyn_class_distribution.items()):
    print(f"Label {label}: {count} samples ({count / len(y_train_adasyn) * 100:.2f}%)")

# Train the Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=5,
    bootstrap=True,
    criterion='entropy',
    class_weight=None,
    random_state=42,
    max_features=0.5,
    n_jobs=-1,
)

# Fit the model on binary data
rf_model.fit(X_train_adasyn, y_train_adasyn)

# Make predictions on training data
y_train_pred = rf_model.predict(X_train_adasyn)

# Make predictions on test data
y_test_pred = rf_model.predict(X_test_selected)

y_validation_pred = rf_model.predict(X_validation_selected)

# Evaluate the model on test data
print("\nClassification Report (Test Data):")
print(classification_report(y_test_binary, y_test_pred, zero_division=0))

# Calculate performance metrics
test_accuracy = accuracy_score(y_test_binary, y_test_pred)
test_balanced_acc = balanced_accuracy_score(y_test_binary, y_test_pred)
train_accuracy = accuracy_score(y_train_adasyn, y_train_pred)
train_balanced_acc = balanced_accuracy_score(y_train_adasyn, y_train_pred)

print(f"\nTrain Accuracy: {train_accuracy:.3f}, Train Balanced Accuracy: {train_balanced_acc:.3f}")
print(f"Test Accuracy: {test_accuracy:.3f}, Test Balanced Accuracy: {test_balanced_acc:.3f}")


# Function to create and plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, title, ax):
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm_norm = np.nan_to_num(cm_norm)  # Replace NaN with 0

    sns.heatmap(cm_norm, annot=True, fmt='.3f', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1], ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')


# Create a figure with two subplots for the confusion matrices
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# Plot confusion matrices for training and test data
plot_confusion_matrix(y_train_adasyn, y_train_pred, 'Training Data Confusion Matrix', ax1)
plot_confusion_matrix(y_test_binary, y_test_pred, 'Test Data Confusion Matrix', ax2)

plt.tight_layout()
plt.show()


In [ ]:
hard_spots_test_index = y_test_pred == 1
hard_spots_test = X_test_new_nz[hard_spots_test_index]
y_test_hard = y_test[hard_spots_test_index]

hard_spots_val_index = y_validation_pred == 1
hard_spots_validation = X_val_new_nz[hard_spots_val_index]
y_validation_hard = y_validation[hard_spots_val_index]

# y_train_pred = rf_model.predict(X_train[feature_list])
hard_spots_train_index = y_train != 0
hard_spots_train = X_train_nz[hard_spots_train_index]
y_train_hard = y_train[hard_spots_train_index]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN

import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt


def reverse_weighted_smooth(
        df,
        features_to_smooth,
        bandwidth=4.0,
        center_emphasis=3.0,
        eps=2.0,
        min_samples=10
):
    """
    Smooth each feature in `features_to_smooth` by Gaussian-weighted averaging,
    but boost the DBSCAN cluster-center’s weight by `center_emphasis` in every
    point’s smoothing computation.
    """
    df_smoothed = df.copy()
    coords = df[['posx', 'posy']].values

    # 1) DBSCAN clustering
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
    labels = clustering.labels_
    unique_labels = [lbl for lbl in np.unique(labels) if lbl != -1]

    # 2) Identify cluster centers
    center_indices = []
    center_indices_by_label = {}
    for lbl in unique_labels:
        mask = (labels == lbl)
        pts = coords[mask]
        centroid = pts.mean(axis=0)
        # find the member closest to centroid
        distances = ((pts - centroid) ** 2).sum(axis=1)
        member_indices = np.where(mask)[0]
        center_idx = member_indices[np.argmin(distances)]
        center_indices.append(center_idx)
        center_indices_by_label[lbl] = center_idx

    # 3) Smooth each feature
    for feature in features_to_smooth:
        if feature not in df.columns or feature in ['posx', 'posy', 'label']:
            continue

        orig = df[feature].values
        sm = np.empty_like(orig, dtype=float)

        for i in range(len(coords)):
            # distances from point i to all points
            dists = np.linalg.norm(coords - coords[i], axis=1)
            w = np.exp(-0.5 * (dists / bandwidth) ** 2)

            lbl = labels[i]
            # if it belongs to a real cluster, boost its center’s weight
            if lbl != -1:
                center_idx = center_indices_by_label[lbl]
                w[center_idx] *= center_emphasis

            # normalize & compute weighted average
            sm[i] = (w * orig).sum() / w.sum()

        df_smoothed[feature] = sm

    return df_smoothed, center_indices, labels


def visualize_clusters_and_centers(df, center_indices, labels, ax, title):
    """
    Scatterplot of (posx,posy) colored by DBSCAN label, with X’s at centers.
    """
    unique_labels = np.unique(labels)
    colors = plt.cm.tab20(np.linspace(0, 1, len(unique_labels)))

    for idx, lbl in enumerate(unique_labels):
        mask = (labels == lbl)
        color = 'black' if lbl == -1 else colors[idx]
        ax.scatter(
            df.loc[mask, 'posx'], df.loc[mask, 'posy'],
            c=[color], edgecolor='k', s=50, alpha=0.6
        )

    centers = df.iloc[center_indices][['posx', 'posy']].values
    ax.scatter(
        centers[:, 0], centers[:, 1],
        c='red', marker='X', s=200, edgecolor='k'
    )
    ax.set_title(f'{title} with Cluster Centers')
    ax.set_xlabel('posx')
    ax.set_ylabel('posy')


# --- USAGE EXAMPLE ---

# Suppose `hard_spots_train` and `hard_spots_test` are your DataFrames,
# and `feature_list` is e.g. list(df.columns).

features_to_smooth = [f for f in feature_list if f not in ['posx', 'posy', 'label']]

# Train set smoothing
hard_spots_train_smoothed, train_center_idxs, train_labels = reverse_weighted_smooth(
    hard_spots_train,
    features_to_smooth,
    bandwidth=2.0,
    center_emphasis=2.0,  # ↑ increase for stronger central pull
    eps=2.0,
    min_samples=10
)

# Test set smoothing
hard_spots_test_smoothed, test_center_idxs, test_labels = reverse_weighted_smooth(
    hard_spots_test,
    features_to_smooth,
    bandwidth=2.0,
    center_emphasis=2.0,
    eps=2.0,
    min_samples=10
)

hard_spots_validation_smoothed, validation_center_idxs, validation_labels = reverse_weighted_smooth(
    hard_spots_validation,
    features_to_smooth,
    bandwidth=2.0,
    center_emphasis=2.0,
    eps=2.0,
    min_samples=10
)



# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
visualize_clusters_and_centers(
    hard_spots_train_smoothed, train_center_idxs, train_labels, axes[0], "Train"
)

visualize_clusters_and_centers(
    hard_spots_test_smoothed, test_center_idxs, test_labels, axes[1], "Train: Smoothed"
)
plt.tight_layout()
plt.show()


# # Visualize the clusters and centers7
# fig, axes = plt.subplots(1, 2, figsize=(14, 6))
# visualize_clusters_and_centers(hard_spots_test, center_indices, labels, axes[1], "test")
# visualize_clusters_and_centers(hard_spots_train, center_indices2, labels2, axes[0], "train")
#

# Plot the smoothing effect for comparison
def plot_smoothing_effect(test_original, test_smooth, train_original, train_smoothed, feature):
    fig, axes = plt.subplots(2, 2, figsize=(8, 8))

    # Plot original values
    sc1 = axes[1][0].scatter(test_original['posx'], test_original['posy'],
                             c=test_original[feature], cmap='viridis',
                             s=50, alpha=0.8)
    axes[1][0].set_title(f"Test: Original {feature}")
    axes[1][0].set_xlabel("posx")
    axes[1][0].set_ylabel("posy")

    sc1 = axes[0][0].scatter(train_original['posx'], train_original['posy'],
                             c=train_original[feature], cmap='viridis',
                             s=50, alpha=0.8)
    axes[0][0].set_title(f"Train: Original {feature}")
    axes[0][0].set_xlabel("posx")
    axes[0][0].set_ylabel("posy")

    # Plot smoothed values
    sc2 = axes[1][1].scatter(test_smooth['posx'], test_smooth['posy'],
                             c=test_smooth[feature], cmap='viridis',
                             s=50, alpha=0.8)
    axes[1][1].set_title(f"Test: Smoothed {feature}")
    axes[1][1].set_xlabel("posx")
    axes[1][1].set_ylabel("posy")

    sc2 = axes[0][1].scatter(train_smoothed['posx'], train_smoothed['posy'],
                             c=train_smoothed[feature], cmap='viridis',
                             s=50, alpha=0.8)
    axes[0][1].set_title(f"Train: Smoothed {feature}")
    axes[0][1].set_xlabel("posx")
    axes[0][1].set_ylabel("posy")

    plt.tight_layout()
    plt.show()


for feature in features_to_smooth:
    plot_smoothing_effect(hard_spots_test, hard_spots_test_smoothed, hard_spots_train, hard_spots_train_smoothed,
                          feature)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import silhouette_score
from tqdm import tqdm


def find_best_smoothing_params(
        df,
        features_to_smooth,
        class_label_col,
        bandwidth_values,
        center_emphasis_values,
        eps=2.0,
        min_samples=10
):
    """
    For each (bw,ce) in the grid, smooth the df, then compute
    silhouette_score on the smoothed feature matrix (only the smoothed
    features), using the true class labels in `class_label_col`.
    Returns a DataFrame of scores and the best (bw, ce).
    """
    results = []
    true_labels = df[class_label_col].values

    for bw in tqdm(bandwidth_values, desc="bandwidth"):
        for ce in center_emphasis_values:
            # 1) smooth
            sm_df, _, _ = reverse_weighted_smooth(
                df, features_to_smooth,
                bandwidth=bw,
                center_emphasis=ce
            )

            X = sm_df[features_to_smooth].values

            # 2) silhouette (note: requires at least 2 points per class)
            try:
                score = silhouette_score(X, true_labels)
            except ValueError:
                score = np.nan

            results.append({
                'bandwidth': bw,
                'center_emphasis': ce,
                'silhouette': score
            })

    results_df = pd.DataFrame(results)
    # pick the best
    best = results_df.loc[results_df['silhouette'].idxmax()]
    return results_df, best['bandwidth'], best['center_emphasis'], best['silhouette']


bw_vals = [7, 8, 9, 10, 11, 12, 13]
ce_vals = [0, 1.0, 2.0, 3]
features = [f for f in feature_list if f not in ['posx', 'posy', 'label', 'class']]

df_all = pd.concat([hard_spots_train, hard_spots_validation], axis=0, ignore_index=True)
df_all['class'] = pd.concat([y_train, y_validation_hard], axis=0, ignore_index=True)

# df_all = hard_spots_validation
# df_all['class'] = y_validation_hard

results_df, best_bw, best_ce, best_score = find_best_smoothing_params(
    df=df_all,
    features_to_smooth=features,
    class_label_col='class',
    bandwidth_values=bw_vals,
    center_emphasis_values=ce_vals
)

print("Best params:", best_bw, best_ce, "– silhouette:", best_score)

In [ ]:
plot_features(hard_spots_train_smoothed, hard_spots_test_smoothed, feature_list)

In [ ]:
feature_list = ['Stiffness',
 'Upstroke',
 'Downstroke',
 'time_to_max',
 'loading_unloading_area_ratio',
 'cubic_coefficient',
 'quartic_coefficient',
 'loading_nonlinearity' ]#, 'force_ratio_75_25']

### Full Classifier

In [ ]:
from plotter import plot_confusion_matrix

# test_index = y_test_pred == 1
# X_test_new_nz = X_test_new[test_index]
# y_test_nz = y_test[test_index]
#
# train_index = y_train != 0
# X_train_nz = X_train[train_index]
# y_train_nz = y_train[train_index]
#
# X_train_selected = X_train_nz[feature_list]
# X_test_selected = X_test_new_nz[feature_list]  #X_test  --- X_test_new

X_train_selected = hard_spots_train_smoothed[feature_list]
X_test_selected = hard_spots_test_smoothed[feature_list]
y_train_nz = y_train_hard
y_test_nz = y_test_hard

# Apply ADASYN for adaptive oversampling
# adasyn = ADASYN(sampling_strategy='auto', random_state=42, n_neighbors=5)
# X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_selected, y_train)

X_train_adasyn, y_train_adasyn = X_train_selected, y_train_nz

# Display new class distribution after ADASYN
adasyn_class_distribution = Counter(y_train_adasyn)
print("\nClass distribution after ADASYN:")
for label, count in sorted(adasyn_class_distribution.items()):
    print(f"Label {label}: {count} samples ({count / len(y_train_adasyn) * 100:.2f}%)")

# Train the Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=5,
    bootstrap=True,
    criterion='entropy',
    class_weight=None,
    random_state=42,
    max_features=0.5,
    n_jobs=-1,
)

# Fit the model
rf_model.fit(X_train_adasyn, y_train_adasyn)

# Make predictions on training data
y_train_pred = rf_model.predict(X_train_adasyn)

# Make predictions on test data
y_test_pred_nz = rf_model.predict(X_test_selected)

# Evaluate the model on test data
print("\nClassification Report (Test Data):")
print(classification_report(y_test_nz, y_test_pred_nz, zero_division=0))

# Calculate performance metrics
test_accuracy = accuracy_score(y_test_nz, y_test_pred_nz)
test_balanced_acc = balanced_accuracy_score(y_test_nz, y_test_pred_nz)
train_accuracy = accuracy_score(y_train_adasyn, y_train_pred)
train_balanced_acc = balanced_accuracy_score(y_train_adasyn, y_train_pred)

# Create a figure with two subplots for the confusion matrices
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# Plot confusion matrices for training and test data
plot_confusion_matrix(y_train_adasyn, y_train_pred, 'Training Data Confusion Matrix', ax1)
plot_confusion_matrix(y_test_nz, y_test_pred_nz, 'Test Data Confusion Matrix', ax2)

plt.tight_layout()
plt.show()


In [ ]:
from plotter import plot_confusion_matrix

test_index = y_test_pred == 1
X_test_new_nz = X_test_scaled[test_index]
y_test_nz = y_test[test_index]

train_index = y_train != 0
X_train_nz = X_train_scaled[train_index]
y_train_nz = y_train[train_index]

X_train_selected = X_train_nz[feature_list]
X_test_selected = X_test_new_nz[feature_list]  #X_test  --- X_test_new

# X_train_selected = hard_spots_train_smoothed[feature_list]
# X_test_selected = hard_spots_test_smoothed[feature_list]
# y_train_nz = y_train_hard
# y_test_nz = y_test_hard

# Apply ADASYN for adaptive oversampling
# adasyn = ADASYN(sampling_strategy='auto', random_state=42, n_neighbors=5)
# X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_selected, y_train)

X_train_adasyn, y_train_adasyn = X_train_selected, y_train_nz

# Display new class distribution after ADASYN
adasyn_class_distribution = Counter(y_train_adasyn)
print("\nClass distribution after ADASYN:")
for label, count in sorted(adasyn_class_distribution.items()):
    print(f"Label {label}: {count} samples ({count / len(y_train_adasyn) * 100:.2f}%)")

# Train the Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=5,
    bootstrap=True,
    criterion='entropy',
    class_weight=None,
    random_state=42,
    max_features=0.5,
    n_jobs=-1,
)

# Fit the model
rf_model.fit(X_train_adasyn, y_train_adasyn)

# Make predictions on training data
y_train_pred = rf_model.predict(X_train_adasyn)

# Make predictions on test data
y_test_pred_nz = rf_model.predict(X_test_selected)

# Evaluate the model on test data
print("\nClassification Report (Test Data):")
print(classification_report(y_test_nz, y_test_pred_nz, zero_division=0))

# Calculate performance metrics
test_accuracy = accuracy_score(y_test_nz, y_test_pred_nz)
test_balanced_acc = balanced_accuracy_score(y_test_nz, y_test_pred_nz)
train_accuracy = accuracy_score(y_train_adasyn, y_train_pred)
train_balanced_acc = balanced_accuracy_score(y_train_adasyn, y_train_pred)

# Create a figure with two subplots for the confusion matrices
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# Plot confusion matrices for training and test data
plot_confusion_matrix(y_train_adasyn, y_train_pred, 'Training Data Confusion Matrix', ax1)
plot_confusion_matrix(y_test_nz, y_test_pred_nz, 'Test Data Confusion Matrix', ax2)

plt.tight_layout()
plt.show()


In [ ]:
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB
# from sklearn.neural_network import MLPClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
#
# from xgboost import XGBClassifier
#
#
#
# # Define models in a list of tuples (name, model instance)
# models = [
#     # ('SVM', SVC(kernel='rbf', C=0.5, gamma='scale', probability=True, random_state=42)),  # n_jobs not supported
#     # ('KNN', KNeighborsClassifier(n_neighbors=8, metric='minkowski', p=2, n_jobs=-1)),
#     # ('Logistic Regression', LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)),
#     # ('Gradient Boosting', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)),
#     # ('Naive Bayes', GaussianNB()),  # no n_jobs
#     ('XGBoost', XGBClassifier(objective='multi:softprob', eval_metric='mlogloss', n_jobs=-1, random_state=42))
# ]
#
#
# def fit_and_evaluate_models(models, X_train, y_train, X_test, y_test):
#     results = []
#
#     for i, (name, model) in enumerate(models):
#         print(f"\nTraining {name}...")
#         model.fit(X_train, y_train - 1  )
#
#         y_train_pred = model.predict(X_train)
#         y_test_pred = model.predict(X_test)
#
#         # Evaluation metrics
#         train_acc = accuracy_score(y_train, y_train_pred)
#         test_acc = accuracy_score(y_test, y_test_pred)
#         train_bal_acc = balanced_accuracy_score(y_train, y_train_pred)
#         test_bal_acc = balanced_accuracy_score(y_test, y_test_pred)
#
#         print(f"{name} Training Accuracy: {train_acc:.4f}")
#         print(f"{name} Training Balanced Accuracy: {train_bal_acc:.4f}")
#         print(f"{name} Test Accuracy: {test_acc:.4f}")
#         print(f"{name} Test Balanced Accuracy: {test_bal_acc:.4f}")
#
#         results.append({
#             'Model': name,
#             'Train Accuracy': train_acc,
#             'Train Balanced Accuracy': train_bal_acc,
#             'Test Accuracy': test_acc,
#             'Test Balanced Accuracy': test_bal_acc
#         })
#
#         # Plot for this model
#         fig, axes = plt.subplots(1, 2, figsize=(12, 5))
#         plot_confusion_matrix(y_train, y_train_pred, f'{name} - Train', axes[0])
#         plot_confusion_matrix(y_test, y_test_pred, f'{name} - Test', axes[1])
#         plt.tight_layout()
#         plt.show()
#
#     return results
#
#
# # X_train_selected = hard_spots_train_smoothed[feature_list]
# # X_test_selected = hard_spots_test_smoothed[feature_list]
# # y_train_nz = y_train_hard
# # y_test_nz = y_test_hard
# # X_train_adasyn, y_train_adasyn = X_train_selected, y_train_nz
#
# # Example call
# results = fit_and_evaluate_models(models, X_train_adasyn, y_train_adasyn, X_test_selected, y_test_nz)
#
#
# # Plot comparison bar chart
# def plot_model_comparison(results):
#     import pandas as pd
#     import seaborn as sns
#
#     df = pd.DataFrame(results)
#     df_melted = df.melt(id_vars='Model', var_name='Metric', value_name='Score')
#
#     plt.figure(figsize=(12, 6))
#     sns.barplot(data=df_melted, x='Model', y='Score', hue='Metric')
#     plt.title('Model Comparison')
#     plt.ylim(0, 1.05)
#     plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
#     plt.tight_layout()
#     plt.show()
#
#
# plot_model_comparison(results)
